In [0]:
# --------------------------------
# Silver Layer - Medallion Architecture (Service Principal Auth)
# --------------------------------

# Fetch Service Principal credentials from Key Vault
client_id = dbutils.secrets.get(scope="capstonescope", key="sp-client-id")
client_secret = dbutils.secrets.get(scope="capstonescope", key="sp-client-secret")
tenant_id = dbutils.secrets.get(scope="capstonescope", key="sp-tenant-id")

storage_account = "capstoneblobsa"
bronze_container = "bronze"
silver_container = "silver"

# Configure Spark to use Service Principal OAuth for storage account
spark.conf.set("fs.azure.account.auth.type", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Base paths for Bronze and Silver containers
bronze_base = f"abfss://{bronze_container}@{storage_account}.dfs.core.windows.net/"
silver_base = f"abfss://{silver_container}@{storage_account}.dfs.core.windows.net/"

# Paths to Bronze Delta tables
bronze_house_path = f"{bronze_base}house_price"
bronze_orders_path = f"{bronze_base}sales_orders"
bronze_world_path = f"{bronze_base}world_population"

# Paths to Silver Delta tables
silver_house_path = f"{silver_base}house_price"
silver_orders_path = f"{silver_base}sales_orders"
silver_world_path = f"{silver_base}world_population"

# --------------------------------
# Read Bronze Delta tables
# --------------------------------
df_house_bronze = spark.read.format("delta").load(bronze_house_path)
df_orders_bronze = spark.read.format("delta").load(bronze_orders_path)
df_world_bronze = spark.read.format("delta").load(bronze_world_path)

# --------------------------------
# Silver transformations (light cleaning, structuring, metadata)
# --------------------------------
from pyspark.sql import functions as F
from pyspark.sql.types import MapType, StringType

def add_metadata(df, source_name):
    return (
        df
        .withColumn("ingestion_timestamp", F.current_timestamp())
        .withColumn("ingestion_date", F.to_date(F.current_timestamp()))
        .withColumn("source_file", F.input_file_name())
        .withColumn("record_source", F.lit(source_name))
    )

# House Price transformation
df_house_silver = (
    df_house_bronze
    .withColumn("value_clean", F.trim(F.col("value")))
    .filter(F.col("value_clean").isNotNull() & (F.length(F.col("value_clean")) > 0))
    .withColumn("json_map", F.from_json(F.col("value_clean"), MapType(StringType(), StringType())))
    .dropDuplicates(["value_clean"])
)
df_house_silver = add_metadata(df_house_silver, "bronze.house_price")
df_house_silver = df_house_silver.withColumn("is_json", F.col("json_map").isNotNull())

# Sales Orders transformation
df_orders_silver = (
    df_orders_bronze
    .withColumn("value_clean", F.trim(F.col("value")))
    .filter(F.col("value_clean").isNotNull() & (F.length(F.col("value_clean")) > 0))
    .dropDuplicates(["value_clean"])
)
df_orders_silver = add_metadata(df_orders_silver, "bronze.sales_orders")

# World Population transformation
df_world_silver = (
    df_world_bronze
    .withColumn("value_clean", F.trim(F.col("value")))
    .filter(F.col("value_clean").isNotNull() & (F.length(F.col("value_clean")) > 0))
    .dropDuplicates(["value_clean"])
)
df_world_silver = add_metadata(df_world_silver, "bronze.world_population")

# --------------------------------
# Write Silver Delta tables
# --------------------------------
(
    df_house_silver.write.format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")
    .save(silver_house_path)
)

(
    df_orders_silver.write.format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")
    .save(silver_orders_path)
)

(
    df_world_silver.write.format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")
    .save(silver_world_path)
)

# --------------------------------
# Delta Lake time travel examples (Silver)
# --------------------------------
df_house_silver_v0 = spark.read.format("delta").option("versionAsOf", 0).load(silver_house_path)
df_orders_silver_v0 = spark.read.format("delta").option("versionAsOf", 0).load(silver_orders_path)
df_world_silver_v0 = spark.read.format("delta").option("versionAsOf", 0).load(silver_world_path)

print("Silver House Price (version 0):")
display(df_house_silver_v0.limit(5))

print("Silver Sales Orders (version 0):")
display(df_orders_silver_v0.limit(5))

print("Silver World Population (version 0):")
display(df_world_silver_v0.limit(5))

from delta.tables import DeltaTable

house_delta = DeltaTable.forPath(spark, silver_house_path)
orders_delta = DeltaTable.forPath(spark, silver_orders_path)
world_delta = DeltaTable.forPath(spark, silver_world_path)

house_first_ts = house_delta.history().orderBy(F.col("timestamp").asc()).first()["timestamp"]
orders_first_ts = orders_delta.history().orderBy(F.col("timestamp").asc()).first()["timestamp"]
world_first_ts = world_delta.history().orderBy(F.col("timestamp").asc()).first()["timestamp"]

df_house_silver_t0 = spark.read.format("delta").option("timestampAsOf", str(house_first_ts)).load(silver_house_path)
df_orders_silver_t0 = spark.read.format("delta").option("timestampAsOf", str(orders_first_ts)).load(silver_orders_path)
df_world_silver_t0 = spark.read.format("delta").option("timestampAsOf", str(world_first_ts)).load(silver_world_path)

print(f"Silver House Price (timestampAsOf={house_first_ts}):")
display(df_house_silver_t0.limit(5))

print(f"Silver Sales Orders (timestampAsOf={orders_first_ts}):")
display(df_orders_silver_t0.limit(5))

print(f"Silver World Population (timestampAsOf={world_first_ts}):")
display(df_world_silver_t0.limit(5))

print("Silver House Price history:")
house_delta.history().show(truncate=False)

print("Silver Sales Orders history:")
orders_delta.history().show(truncate=False)

print("Silver World Population history:")
world_delta.history().show(truncate=False)

# --------------------------------
# Register Silver tables in SQL database
# --------------------------------
spark.sql("CREATE DATABASE IF NOT EXISTS silver_db")
spark.sql(f"CREATE TABLE IF NOT EXISTS silver_db.house_price USING DELTA LOCATION '{silver_house_path}'")
spark.sql(f"CREATE TABLE IF NOT EXISTS silver_db.sales_orders USING DELTA LOCATION '{silver_orders_path}'")
spark.sql(f"CREATE TABLE IF NOT EXISTS silver_db.world_population USING DELTA LOCATION '{silver_world_path}'")

# Sample query
display(spark.sql("SELECT * FROM silver_db.house_price LIMIT 5"))


Silver House Price (version 0):


value,airconditioning,area,basement,bathrooms,bedrooms,furnishingstatus,guestroom,hotwaterheating,mainroad,parking,prefarea,price,stories,value_clean,json_map,ingestion_timestamp,ingestion_date,source_file,record_source,is_json
"{""price"":""10150000"",""area"":""16200"",""bedrooms"":""5"",""bathrooms"":""3"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""no"",""parking"":""0"",""prefarea"":""no"",""furnishingstatus"":""unfurnished""}",null,null,null,null,null,null,null,null,null,null,null,null,null,"{""price"":""10150000"",""area"":""16200"",""bedrooms"":""5"",""bathrooms"":""3"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""no"",""parking"":""0"",""prefarea"":""no"",""furnishingstatus"":""unfurnished""}","Map(bedrooms -> 5, area -> 16200, stories -> 2, price -> 10150000, furnishingstatus -> unfurnished, mainroad -> yes, airconditioning -> no, guestroom -> no, bathrooms -> 3, parking -> 0, basement -> no, hotwaterheating -> no, prefarea -> no)",2025-09-27T05:05:03.620363Z,2025-09-27,,bronze.house_price,true
"{""price"":""10150000"",""area"":""8580"",""bedrooms"":""4"",""bathrooms"":""3"",""stories"":""4"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""semi-furnished""}",null,null,null,null,null,null,null,null,null,null,null,null,null,"{""price"":""10150000"",""area"":""8580"",""bedrooms"":""4"",""bathrooms"":""3"",""stories"":""4"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""semi-furnished""}","Map(bedrooms -> 4, area -> 8580, stories -> 4, price -> 10150000, furnishingstatus -> semi-furnished, mainroad -> yes, airconditioning -> yes, guestroom -> no, bathrooms -> 3, parking -> 2, basement -> no, hotwaterheating -> no, prefarea -> yes)",2025-09-27T05:05:03.620363Z,2025-09-27,,bronze.house_price,true
"{""price"":""10850000"",""area"":""7500"",""bedrooms"":""3"",""bathrooms"":""3"",""stories"":""1"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""semi-furnished""}",null,null,null,null,null,null,null,null,null,null,null,null,null,"{""price"":""10850000"",""area"":""7500"",""bedrooms"":""3"",""bathrooms"":""3"",""stories"":""1"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""semi-furnished""}","Map(bedrooms -> 3, area -> 7500, stories -> 1, price -> 10850000, furnishingstatus -> semi-furnished, mainroad -> yes, airconditioning -> yes, guestroom -> no, bathrooms -> 3, parking -> 2, basement -> yes, hotwaterheating -> no, prefarea -> yes)",2025-09-27T05:05:03.620363Z,2025-09-27,,bronze.house_price,true
"{""price"":""11410000"",""area"":""7420"",""bedrooms"":""4"",""bathrooms"":""1"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""yes"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""no"",""furnishingstatus"":""furnished""}",null,null,null,null,null,null,null,null,null,null,null,null,null,"{""price"":""11410000"",""area"":""7420"",""bedrooms"":""4"",""bathrooms"":""1"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""yes"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""no"",""furnishingstatus"":""furnished""}","Map(bedrooms -> 4, area -> 7420, stories -> 2, price -> 11410000, furnishingstatus -> furnished, mainroad -> yes, airconditioning -> yes, guestroom -> yes, bathrooms -> 1, parking -> 2, basement -> yes, hotwa

Silver Sales Orders (version 0):


value,value_clean,ingestion_timestamp,ingestion_date,source_file,record_source
"1,2025-10-20 00:00:00.0000000,150,""South"",""Furniture"",7700.25","1,2025-10-20 00:00:00.0000000,150,""South"",""Furniture"",7700.25",2025-09-27T05:05:07.409632Z,2025-09-27,,bronze.sales_orders
"10,2025-09-10 00:00:00.0000000,110,""South"",""Electronics"",13400.00","10,2025-09-10 00:00:00.0000000,110,""South"",""Electronics"",13400.00",2025-09-27T05:05:07.409632Z,2025-09-27,,bronze.sales_orders
"11,2025-09-11 00:00:00.0000000,111,""East"",""Furniture"",8900.75","11,2025-09-11 00:00:00.0000000,111,""East"",""Furniture"",8900.75",2025-09-27T05:05:07.409632Z,2025-09-27,,bronze.sales_orders
"12,2025-09-12 00:00:00.0000000,112,""North"",""Office Supplies"",510.00","12,2025-09-12 00:00:00.0000000,112,""North"",""Office Supplies"",510.00",2025-09-27T05:05:07.409632Z,2025-09-27,,bronze.sales_orders
"13,2025-09-13 00:00:00.0000000,113,""West"",""Electronics"",14200.00","13,2025-09-13 00:00:00.0000000,113,""West"",""Electronics"",14200.00",2025-09-27T05:05:07.409632Z,2025-09-27,,bronze.sales_orders


Silver World Population (version 0):


...

value,value_clean,ingestion_timestamp,ingestion_date,source_file,record_source
"1,2025-10-20 00:00:00.0000000,150,""South"",""Furniture"",7700.25","1,2025-10-20 00:00:00.0000000,150,""South"",""Furniture"",7700.25",2025-09-27T05:05:07.409632Z,2025-09-27,,bronze.sales_orders
"10,2025-09-10 00:00:00.0000000,110,""South"",""Electronics"",13400.00","10,2025-09-10 00:00:00.0000000,110,""South"",""Electronics"",13400.00",2025-09-27T05:05:07.409632Z,2025-09-27,,bronze.sales_orders
"11,2025-09-11 00:00:00.0000000,111,""East"",""Furniture"",8900.75","11,2025-09-11 00:00:00.0000000,111,""East"",""Furniture"",8900.75",2025-09-27T05:05:07.409632Z,2025-09-27,,bronze.sales_orders
"12,2025-09-12 00:00:00.0000000,112,""North"",""Office Supplies"",510.00","12,2025-09-12 00:00:00.0000000,112,""North"",""Office Supplies"",510.00",2025-09-27T05:05:07.409632Z,2025-09-27,,bronze.sales_orders
"13,2025-09-13 00:00:00.0000000,113,""West"",""Electronics"",14200.00","13,2025-09-13 00:00:00.0000000,113,""West"",""Electronics"",14200.00",2025-09-27T05:05:07.409632Z,2025-09-27,,bronze.sales_orders


Silver World Population (timestampAsOf=2025-09-27 05:05:10):


value,value_clean,ingestion_timestamp,ingestion_date,source_file,record_source
"""1,CHN,China,Beijing,Asia,1425887337,1424929781,1393715448,1348191368,1264099069,1153704252,982372466,822534450,9706961,146.8933,1,17.88""","""1,CHN,China,Beijing,Asia,1425887337,1424929781,1393715448,1348191368,1264099069,1153704252,982372466,822534450,9706961,146.8933,1,17.88""",2025-09-27T05:05:09.054741Z,2025-09-27,,bronze.world_population
"""10,MEX,Mexico,Mexico City,North America,127504125,125998302,120149897,112532401,97873442,81720428,67705186,50289306,1964375,64.9082,1.0063,1.6""","""10,MEX,Mexico,Mexico City,North America,127504125,125998302,120149897,112532401,97873442,81720428,67705186,50289306,1964375,64.9082,1.0063,1.6""",2025-09-27T05:05:09.054741Z,2025-09-27,,bronze.world_population
"""100,TGO,Togo,Lomé,Africa,8848699,8442580,7473229,6571855,5008035,3875947,2838110,2197383,56785,155.8281,1.0236,0.11""","""100,TGO,Togo,Lomé,Africa,8848699,8442580,7473229,6571855,5008035,3875947,2838110,2197383,56785,155.8281,1.0236,0.11""",2025-09-27T05:05:09.054741Z,2025-09-27,,bronze.world_population
"""101,CHE,Switzerland,Bern,Europe,8740472,8638613,8281732,7822435,7182059,6711693,6319113,6181227,41284,211.7157,1.0056,0.11""","""101,CHE,Switzerland,Bern,Europe,8740472,8638613,8281732,7822435,7182059,6711693,6319113,6181227,41284,211.7157,1.0056,0.11""",2025-09-27T05:05:09.054741Z,2025-09-27,,bronze.world_population
"""102,SLE,Sierra Leone,Freetown,Africa,8605718,8233969,7314773,6436698,4584067,4325388,3367477,2778557,71740,119.957,1.022,0.11""","""102,SLE,Sierra Leone,Freetown,Africa,8605718,8233969,7314773,6436698,4584067,4325388,3367477,2778557,71740,119.957,1.022,0.11""",2025-09-27T05:05:09.054741Z,2025-09-27,,bronze.world_population


Silver House Price history:
+-------+-------------------+---------------+--------------------------------+---------+------------------------------------------------------------+---------------------------------------------------------------------------+------------------+--------------------+-----------+-----------------+-------------+--------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId         |userName                        |operation|operationParameters                                         |job                                                                        |notebook          |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                                                                              |userMetadata|engineInfo                         |
+-------+-------------------+---

value,airconditioning,area,basement,bathrooms,bedrooms,furnishingstatus,guestroom,hotwaterheating,mainroad,parking,prefarea,price,stories,value_clean,json_map,ingestion_timestamp,ingestion_date,source_file,record_source,is_json
"{""price"":""10150000"",""area"":""16200"",""bedrooms"":""5"",""bathrooms"":""3"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""no"",""parking"":""0"",""prefarea"":""no"",""furnishingstatus"":""unfurnished""}",null,null,null,null,null,null,null,null,null,null,null,null,null,"{""price"":""10150000"",""area"":""16200"",""bedrooms"":""5"",""bathrooms"":""3"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""no"",""parking"":""0"",""prefarea"":""no"",""furnishingstatus"":""unfurnished""}","Map(bedrooms -> 5, area -> 16200, stories -> 2, price -> 10150000, furnishingstatus -> unfurnished, mainroad -> yes, airconditioning -> no, guestroom -> no, bathrooms -> 3, parking -> 0, basement -> no, hotwaterheating -> no, prefarea -> no)",2025-09-30T12:18:54.213642Z,2025-09-30,,bronze.house_price,true
"{""price"":""10150000"",""area"":""8580"",""bedrooms"":""4"",""bathrooms"":""3"",""stories"":""4"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""semi-furnished""}",null,null,null,null,null,null,null,null,null,null,null,null,null,"{""price"":""10150000"",""area"":""8580"",""bedrooms"":""4"",""bathrooms"":""3"",""stories"":""4"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""semi-furnished""}","Map(bedrooms -> 4, area -> 8580, stories -> 4, price -> 10150000, furnishingstatus -> semi-furnished, mainroad -> yes, airconditioning -> yes, guestroom -> no, bathrooms -> 3, parking -> 2, basement -> no, hotwaterheating -> no, prefarea -> yes)",2025-09-30T12:18:54.213642Z,2025-09-30,,bronze.house_price,true
"{""price"":""10850000"",""area"":""7500"",""bedrooms"":""3"",""bathrooms"":""3"",""stories"":""1"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""semi-furnished""}",null,null,null,null,null,null,null,null,null,null,null,null,null,"{""price"":""10850000"",""area"":""7500"",""bedrooms"":""3"",""bathrooms"":""3"",""stories"":""1"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""semi-furnished""}","Map(bedrooms -> 3, area -> 7500, stories -> 1, price -> 10850000, furnishingstatus -> semi-furnished, mainroad -> yes, airconditioning -> yes, guestroom -> no, bathrooms -> 3, parking -> 2, basement -> yes, hotwaterheating -> no, prefarea -> yes)",2025-09-30T12:18:54.213642Z,2025-09-30,,bronze.house_price,true
"{""price"":""11410000"",""area"":""7420"",""bedrooms"":""4"",""bathrooms"":""1"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""yes"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""no"",""furnishingstatus"":""furnished""}",null,null,null,null,null,null,null,null,null,null,null,null,null,"{""price"":""11410000"",""area"":""7420"",""bedrooms"":""4"",""bathrooms"":""1"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""yes"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""no"",""furnishingstatus"":""furnished""}","Map(bedrooms -> 4, area -> 7420, stories -> 2, price -> 11410000, furnishingstatus -> furnished, mainroad -> yes, airconditioning -> yes, guestroom -> yes, bathrooms -> 1, parking -> 2, basement -> yes, hotwa